In [17]:
import os
import fileinput as fi
import glob
import moviepy.editor as mp
import pandas as pd


os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())


SyntaxError: invalid syntax (Temp/ipykernel_22964/1787979151.py, line 8)

### Arabic Annotation collection

In [ ]:
df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)

In [55]:
cpt = sum([len(files) for r, d, files in os.walk("D:/02_batch_2/")])
cpt

35070

In [73]:
li = []

try:

    for i,dname in enumerate(df_dirname['Folder']):
        path = r'D:/02_batch_2/'
        files = glob.glob(path + dname + '/*.txt')
        
        for i,f in enumerate(files):

            parse_error = False
            file_not_found = False

            stock = os.path.basename(f)            
            temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
            temp_df['filePath'] = path + dname + '/'
            temp_df['textFileName'] = stock     
            temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
            temp_df['duration'] = temp_df.stop-temp_df.start
            li.append(temp_df)
            #print(f'Successfully created dataframe for {dname} {stock} with shape {temp_df.shape}')

except pd.errors.ParserError:
    parse_error = True
    temp_df = pd.read_csv(f, delimiter="\t",skiprows=4)
    li.append(temp_df)
    print(f'Error for {dname} {stock} with shape {temp_df.shape}')

else:
    li.append(temp_df)


df = pd.concat(li, axis=0)
print(df.shape)
df.head(3)

b'Skipping line 18: expected 5 fields, saw 7\n'
b'Skipping line 12: expected 5 fields, saw 6\n'
b'Skipping line 5: expected 5 fields, saw 8\n'


(507968, 9)


,speaker,gender,start,stop,transcript,filePath,textFileName,audioFileName,duration
0,speakeA,M,8.998978,23.522329,الحقوني الحقوني بطني بطني بموت ما أقدر الحقون...,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,14.523351
1,speakeA,M,23.522329,25.772074,أضحك عليكو بس أختبركو.,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,2.249745
2,speakeB,M,33.546191,38.020683,اختبار ها تمزح معي ها تمزح معي ها.,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,4.474492


In [72]:
df.groupby('filePath').agg({'filePath':{'count'}}).reset_index()

filePath       
                                                count
0             D:/02_batch_2/37_degrees_series/   7881
1                   D:/02_batch_2/Almajd_kids/  28779
2        D:/02_batch_2/Ba3d_altes3een_Podcast/   6389
3                           D:/02_batch_2/C4N/  28828
4                D:/02_batch_2/Dupamicaffeine/    387
5                 D:/02_batch_2/Mojtm3_Ethraa/   8571
6                 D:/02_batch_2/Naif_aljaweni/  11546
7   D:/02_batch_2/Sando9_aswad_turki_alfaisal/   2913
8                D:/02_batch_2/Shabab_Ela3mal/   3016
9       D:/02_batch_2/Update_NewMedia_Podcast/   1789
10             D:/02_batch_2/abdullah_Alabdli/    851
11                   D:/02_batch_2/abo_hamdan/   2605
12                      D:/02_batch_2/ajaweed/   8760
13                D:/02_batch_2/aljwab_alkafi/   6200
14                       D:/02_batch_2/almsar/   2655
15                      D:/02_batch_2/altamas/  10987
16                        D:/02_batch_2/aqdar/    717
17                        D:/02_batch_2/arshz/   1831
18                    D:/02_batch_2/assrarahm/   7411
19                  D:/02_batch_2/belmo5taser/   9712
20                        D:/02_batch_2/brief/   2599
21               D:/02_batch_2/bronze_stories/   2469
22             D:/02_batch_2/buraik_alshalawi/  19512
23                      D:/02_batch_2/ertecaz/   8956
24                       D:/02_batch_2/fenjan/  57266
25                D:/02_batch_2/ghadah_aleidi/   1280
26                       D:/02_batch_2/ghaima/   1432
27                         D:/02_batch_2/heel/  10011
28                D:/02_batch_2/history_yaman/   4752
29                          D:/02_batch_2/hrm/   5681
30                   D:/02_batch_2/jaddo_saif/  10864
31                       D:/02_batch_2/jawlan/   7376
32                  D:/02_batch_2/khalaf_zone/   3859
33            D:/02_batch_2/lail_nahar_series/  10857
34             D:/02_batch_2/liqaa_wali_alahd/    938
35                     D:/02_batch_2/m3_3adel/   3873
36           D:/02_batch_2/mamlakat_althqafat/   1469
37                 D:/02_batch_2/markaz_banon/   2433
38                  D:/02_batch_2/min_alsifer/  28139
39                         D:/02_batch_2/mo7a/  14846
40         D:/02_batch_2/mohammed_almutairi_1/  13940
41         D:/02_batch_2/mohammed_almutairi_2/   5976
42                          D:/02_batch_2/nas/  17620
43                D:/02_batch_2/podcast_elham/   6772
44               D:/02_batch_2/podcast_qissah/   2670
45               D:/02_batch_2/podcast_saraha/   4181
46                 D:/02_batch_2/podcast_taqa/   3270
47                      D:/02_batch_2/qeninah/  16676
48               D:/02_batch_2/qisas_najdeyat/   1504
49                    D:/02_batch_2/qisas_r3b/   5445
50                      D:/02_batch_2/radio_9/   3139
51         D:/02_batch_2/sarah_alabedulkareem/  21896
52              D:/02_batch_2/sawalf_business/  17546
53                       D:/02_batch_2/shamaa/  15476
54                       D:/02_batch_2/sharek/   3223
55  D:/02_batch_2/things_your_doctor_tell_you/   1609
56                D:/02_batch_2/under_the_hat/   8493
57            D:/02_batch_2/universty_leaders/   3590
58            D:/02_batch_2/zainab_almatroodi/   4502

In [51]:
df_filePath_lk = df.groupby('filePath').agg({'duration':sum}).reset_index()
df_filePath_lk['sumTotal'] = df_filePath_lk['duration'].sum()
df_filePath_lk['percent'] = df_filePath_lk.duration.div(df_filePath_lk.sumTotal)
df_filePath_lk['countSeconds'] = df_filePath_lk.duration.mul(df_filePath_lk.percent)
df_filePath_lk.countSeconds.sum()
df_filePath_lk.head(3)


,filePath,duration,sumTotal,percent,countSeconds
0,D:/02_batch_2/37_degrees_series/,52004.323579,2322.995242,22.386754,1.164208e+06
1,D:/02_batch_2/Almajd_kids/,208921.676217,2322.995242,89.936334,1.878965e+07
2,D:/02_batch_2/Ba3d_altes3een_Podcast/,127268.085322,2322.995242,54.786201,6.972535e+06


### Video Clipping

In [78]:
temp = df.iloc[0,5]
my_clip = mp.AudioFileClip(os.path.join(path, temp))
clip = my_clip.subclip(df.iloc[0,2],df.iloc[0,3])
clip.write_audiofile(r"D:\nf\audio\temp.mp3")



MoviePy - Writing audio in D:\nf\audio\temp.mp3


MoviePy - Done.


#### Directory Structure

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [ ]:
list_files(r"D:\02_batch_2")